In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
!ls /content/drive/MyDrive


'Colab Notebooks'
 dataset.zip
'Google AI Studio'
 IMG-20250612-WA0000.jpg
 IMG-20251008-WA0008.jpg
'Machine Learning Training - Certificate of Completion (1).pdf'
 PHOTO_1_11zon.jpg
'RB LEVEL 1 C.pdf'
'RB LEVEL 2 C.pdf'
'Ruturaj Bhujbal(Core).pdf'
 VID_20250414_141757.mp4


In [6]:
!unzip "/content/drive/MyDrive/dataset.zip"


Archive:  /content/drive/MyDrive/dataset.zip
   creating: dataset/
   creating: dataset/Center/
  inflating: dataset/Center/0.png    
  inflating: dataset/Center/1.png    
  inflating: dataset/Center/10.png   
  inflating: dataset/Center/100.png  
  inflating: dataset/Center/101.png  
  inflating: dataset/Center/102.png  
  inflating: dataset/Center/103.png  
  inflating: dataset/Center/104.png  
  inflating: dataset/Center/105.png  
  inflating: dataset/Center/106.png  
  inflating: dataset/Center/107.png  
  inflating: dataset/Center/108.png  
  inflating: dataset/Center/109.png  
  inflating: dataset/Center/11.png   
  inflating: dataset/Center/110.png  
  inflating: dataset/Center/111.png  
  inflating: dataset/Center/112.png  
  inflating: dataset/Center/113.png  
  inflating: dataset/Center/114.png  
  inflating: dataset/Center/115.png  
  inflating: dataset/Center/116.png  
  inflating: dataset/Center/117.png  
  inflating: dataset/Center/118.png  
  inflating: dataset/Center/11

In [7]:
!ls dataset


Center	Donut  Edge-Loc  Edge-Ring  Loc  Random  Scratch


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Prepare data
data = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train = data.flow_from_directory(
    "dataset",
    target_size=(224,224),
    subset="training"
)

val = data.flow_from_directory(
    "dataset",
    target_size=(224,224),
    subset="validation"
)

# Load MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

# Add classifier
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(train.num_classes, activation='softmax')(x)

model = tf.keras.Model(base_model.input, output)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train
model.fit(train, validation_data=val, epochs=5)


Found 672 images belonging to 7 classes.
Found 168 images belonging to 7 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 34s 842ms/step - accuracy: 0.1942 - loss: 2.0962 - val_accuracy: 0.4464 - val_loss: 1.5557
Epoch 2/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 170ms/step - accuracy: 0.4866 - loss: 1.4892 - val_accuracy: 0.5476 - val_loss: 1.3856
Epoch 3/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 136ms/step - accuracy: 0.5190 - loss: 1.3287 - val_accuracy: 0.5536 - val_loss: 1.2767
Epoch 4/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 131ms/step - accuracy: 0.5785 - loss: 1.1892 - val_accuracy: 0.5476 - val_loss: 1.2133
Epoch 5/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 123ms/step - accuracy: 0.6318 - loss: 1.0838 - val_accuracy: 0.5417 - val_loss: 1.1848


In [9]:
# Unfreeze some layers
base_model.trainable = True

# freeze first 100 layers only
for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(train, validation_data=val, epochs=5)


Epoch 1/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 32s 514ms/step - accuracy: 0.1844 - loss: 3.4211 - val_accuracy: 0.5536 - val_loss: 1.1964
Epoch 2/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 4s 188ms/step - accuracy: 0.2837 - loss: 2.5100 - val_accuracy: 0.5655 - val_loss: 1.2161
Epoch 3/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 135ms/step - accuracy: 0.3885 - loss: 1.7458 - val_accuracy: 0.5238 - val_loss: 1.2509
Epoch 4/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 135ms/step - accuracy: 0.4803 - loss: 1.3960 - val_accuracy: 0.5000 - val_loss: 1.2891
Epoch 5/5
21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 129ms/step - accuracy: 0.5754 - loss: 1.2071 - val_accuracy: 0.5119 - val_loss: 1.3169


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train = data.flow_from_directory(
    "dataset",
    target_size=(224,224),
    subset="training"
)

val = data.flow_from_directory(
    "dataset",
    target_size=(224,224),
    subset="validation"
)


Found 672 images belonging to 7 classes.
Found 168 images belonging to 7 classes.


In [12]:
model.fit(train, validation_data=val, epochs=8)


Epoch 1/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 12s 592ms/step - accuracy: 0.3753 - loss: 1.8187 - val_accuracy: 0.3214 - val_loss: 2.4342
Epoch 2/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 11s 524ms/step - accuracy: 0.4440 - loss: 1.4716 - val_accuracy: 0.3214 - val_loss: 2.2987
Epoch 3/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 11s 507ms/step - accuracy: 0.5184 - loss: 1.3177 - val_accuracy: 0.3095 - val_loss: 2.2696
Epoch 4/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 11s 553ms/step - accuracy: 0.4922 - loss: 1.3210 - val_accuracy: 0.3393 - val_loss: 2.2117
Epoch 5/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 12s 557ms/step - accuracy: 0.5339 - loss: 1.2463 - val_accuracy: 0.3750 - val_loss: 2.1767
Epoch 6/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 12s 559ms/step - accuracy: 0.5549 - loss: 1.1969 - val_accuracy: 0.3214 - val_loss: 2.0783
Epoch 7/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 12s 557ms/step - accuracy: 0.5776 - loss: 1.1666 - val_accuracy: 0.3512 - val_loss: 2.1308
Epoch 8/8
21/21 ━━━━━━━━━━━━━━━━━━━━ 12s 555ms/step - accuracy: 0.5557 - loss: 1.1526 - val_accuracy: 0.

In [13]:
import shutil
import os

keep = ["Center","Donut","Scratch","Edge-Ring"]

for folder in os.listdir("dataset"):
    if folder not in keep:
        shutil.rmtree(f"dataset/{folder}")


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=25,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train = data.flow_from_directory(
    "dataset",
    target_size=(224,224),
    subset="training"
)

val = data.flow_from_directory(
    "dataset",
    target_size=(224,224),
    subset="validation"
)


Found 384 images belonging to 4 classes.
Found 96 images belonging to 4 classes.


In [15]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(train.num_classes, activation='softmax')(x)

model = tf.keras.Model(base_model.input, output)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [16]:
model.fit(train, validation_data=val, epochs=10)


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.4563 - loss: 1.2855 - val_accuracy: 0.6562 - val_loss: 0.9140
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 621ms/step - accuracy: 0.6929 - loss: 0.8071 - val_accuracy: 0.6354 - val_loss: 0.9451
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 494ms/step - accuracy: 0.7653 - loss: 0.6444 - val_accuracy: 0.6771 - val_loss: 0.8990
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 629ms/step - accuracy: 0.7161 - loss: 0.7264 - val_accuracy: 0.7396 - val_loss: 0.7150
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 494ms/step - accuracy: 0.7738 - loss: 0.6191 - val_accuracy: 0.6979 - val_loss: 0.7197
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 628ms/step - accuracy: 0.7951 - loss: 0.5258 - val_accuracy: 0.8125 - val_loss: 0.5684
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 500ms/step - accuracy: 0.8066 - loss: 0.5228 - val_accuracy: 0.7292 - val_loss: 0.6954
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 618ms/step - accuracy: 0.8184 - loss: 0.4987 - val_accuracy: 0.77

In [17]:
model.fit(train, validation_data=val, epochs=5)


Epoch 1/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 578ms/step - accuracy: 0.8382 - loss: 0.4406 - val_accuracy: 0.7188 - val_loss: 0.6732
Epoch 2/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 627ms/step - accuracy: 0.8261 - loss: 0.4165 - val_accuracy: 0.7292 - val_loss: 0.6913
Epoch 3/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 498ms/step - accuracy: 0.8232 - loss: 0.4187 - val_accuracy: 0.7396 - val_loss: 0.6855
Epoch 4/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 620ms/step - accuracy: 0.8316 - loss: 0.4526 - val_accuracy: 0.7604 - val_loss: 0.6645
Epoch 5/5
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 490ms/step - accuracy: 0.8454 - loss: 0.4056 - val_accuracy: 0.8021 - val_loss: 0.5354


In [18]:
# Unfreeze last layers
base_model.trainable = True

for layer in base_model.layers[:-30]:
    layer.trainable = False


In [19]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=30,
    zoom_range=0.35,
    width_shift_range=0.25,
    height_shift_range=0.25,
    brightness_range=[0.8,1.2],
    horizontal_flip=True
)

train = data.flow_from_directory(
    "dataset",
    target_size=(224,224),
    subset="training"
)

val = data.flow_from_directory(
    "dataset",
    target_size=(224,224),
    subset="validation"
)


Found 384 images belonging to 4 classes.
Found 96 images belonging to 4 classes.


In [21]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=3,
    restore_best_weights=True
)


In [22]:
model.fit(
    train,
    validation_data=val,
    epochs=15,
    callbacks=[callback]
)


Epoch 1/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.4474 - loss: 2.5796 - val_accuracy: 0.7083 - val_loss: 0.7701
Epoch 2/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 692ms/step - accuracy: 0.5150 - loss: 1.8111 - val_accuracy: 0.7812 - val_loss: 0.6018
Epoch 3/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 567ms/step - accuracy: 0.5700 - loss: 1.3487 - val_accuracy: 0.7500 - val_loss: 0.7635
Epoch 4/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 686ms/step - accuracy: 0.6568 - loss: 0.9832 - val_accuracy: 0.7292 - val_loss: 0.8544
Epoch 5/15
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 567ms/step - accuracy: 0.7096 - loss: 0.7817 - val_accuracy: 0.7188 - val_loss: 0.8177


In [23]:
train = data.flow_from_directory(
    "dataset",
    target_size=(256,256),
    subset="training"
)

val = data.flow_from_directory(
    "dataset",
    target_size=(256,256),
    subset="validation"
)


Found 384 images belonging to 4 classes.
Found 96 images belonging to 4 classes.


In [25]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(256,256,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(train.num_classes, activation='softmax')(x)

model = tf.keras.Model(base_model.input, output)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


/tmp/ipython-input-514573126.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(


In [26]:
model.fit(train, validation_data=val, epochs=10)


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.3789 - loss: 1.5952 - val_accuracy: 0.6562 - val_loss: 0.9617
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 774ms/step - accuracy: 0.6613 - loss: 0.8982 - val_accuracy: 0.6354 - val_loss: 0.9103
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 707ms/step - accuracy: 0.6731 - loss: 0.8534 - val_accuracy: 0.7188 - val_loss: 0.7871
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 820ms/step - accuracy: 0.7315 - loss: 0.6999 - val_accuracy: 0.7292 - val_loss: 0.7658
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 811ms/step - accuracy: 0.7467 - loss: 0.6565 - val_accuracy: 0.7188 - val_loss: 0.7238
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 689ms/step - accuracy: 0.7661 - loss: 0.6120 - val_accuracy: 0.7604 - val_loss: 0.6867
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 826ms/step - accuracy: 0.8033 - loss: 0.5848 - val_accuracy: 0.7812 - val_loss: 0.7224
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 820ms/step - accuracy: 0.7567 - loss: 0.6557 - val_accuracy: 

In [27]:
class_weights = {
    0:1.0,
    1:1.2,
    2:1.2,
    3:1.3
}

model.fit(
    train,
    validation_data=val,
    epochs=10,
    class_weight=class_weights
)


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 866ms/step - accuracy: 0.7981 - loss: 0.6154 - val_accuracy: 0.7708 - val_loss: 0.6819
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 711ms/step - accuracy: 0.7702 - loss: 0.6592 - val_accuracy: 0.8021 - val_loss: 0.5536
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 764ms/step - accuracy: 0.7845 - loss: 0.5854 - val_accuracy: 0.6979 - val_loss: 0.7471
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 820ms/step - accuracy: 0.7880 - loss: 0.6548 - val_accuracy: 0.8021 - val_loss: 0.6601
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 767ms/step - accuracy: 0.7552 - loss: 0.6622 - val_accuracy: 0.7500 - val_loss: 0.7551
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 702ms/step - accuracy: 0.7959 - loss: 0.5908 - val_accuracy: 0.7708 - val_loss: 0.7323
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 831ms/step - accuracy: 0.8017 - loss: 0.5568 - val_accuracy: 0.7604 - val_loss: 0.6566
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 821ms/step - accuracy: 0.7749 - loss: 0.6182 - val_accuracy

In [28]:
train = data.flow_from_directory(
    "dataset",
    target_size=(224,224),
    color_mode="grayscale",
    subset="training"
)

val = data.flow_from_directory(
    "dataset",
    target_size=(224,224),
    color_mode="grayscale",
    subset="validation"
)


Found 384 images belonging to 4 classes.
Found 96 images belonging to 4 classes.


In [29]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32,3,activation='relu',input_shape=(224,224,1)),
    layers.MaxPooling2D(),
    layers.Conv2D(64,3,activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128,3,activation='relu'),
    layers.GlobalAveragePooling2D(),
    layers.Dense(64,activation='relu'),
    layers.Dense(4,activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(train, validation_data=val, epochs=10)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 10s 379ms/step - accuracy: 0.2295 - loss: 1.3898 - val_accuracy: 0.2500 - val_loss: 1.3855
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 360ms/step - accuracy: 0.2549 - loss: 1.3851 - val_accuracy: 0.2500 - val_loss: 1.3857
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 261ms/step - accuracy: 0.2726 - loss: 1.3844 - val_accuracy: 0.2500 - val_loss: 1.3852
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 258ms/step - accuracy: 0.2474 - loss: 1.3840 - val_accuracy: 0.2500 - val_loss: 1.3840
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 258ms/step - accuracy: 0.2443 - loss: 1.3833 - val_accuracy: 0.2396 - val_loss: 1.3827
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 354ms/step - accuracy: 0.2501 - loss: 1.3834 - val_accuracy: 0.2500 - val_loss: 1.3827
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 263ms/step - accuracy: 0.2157 - loss: 1.3835 - val_accuracy: 0.2812 - val_loss: 1.3841
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 265ms/step - accuracy: 0.2214 - loss: 1.3776 - val_accuracy: 0

In [30]:
val_loss, val_acc = model.evaluate(val)
print("Validation Accuracy:", val_acc)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 195ms/step - accuracy: 0.3529 - loss: 1.3697
Validation Accuracy: 0.3541666567325592


In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=25,
    zoom_range=0.3,
    horizontal_flip=True
)

train = data.flow_from_directory(
    "dataset",
    target_size=(224,224),
    subset="training"
)

val = data.flow_from_directory(
    "dataset",
    target_size=(224,224),
    subset="validation"
)


Found 384 images belonging to 4 classes.
Found 96 images belonging to 4 classes.


In [32]:
import tensorflow as tf

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output = tf.keras.layers.Dense(train.num_classes, activation='softmax')(x)

model = tf.keras.Model(base_model.input, output)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [33]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "best_model.h5",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max"
)

model.fit(
    train,
    validation_data=val,
    epochs=10,
    callbacks=[checkpoint]
)


Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step - accuracy: 0.4465 - loss: 1.2830

12/12 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.4558 - loss: 1.2684 - val_accuracy: 0.6250 - val_loss: 0.9546
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step - accuracy: 0.6543 - loss: 0.8936

12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 517ms/step - accuracy: 0.6574 - loss: 0.8905 - val_accuracy: 0.6354 - val_loss: 0.8878
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step - accuracy: 0.7374 - loss: 0.7430

12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 630ms/step - accuracy: 0.7369 - loss: 0.7430 - val_accuracy: 0.6771 - val_loss: 0.8708
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - accuracy: 0.8153 - loss: 0.5365

12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 509ms/step - accuracy: 0.8145 - loss: 0.5375 - val_accuracy: 0.7500 - val_loss: 0.7121
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step - accuracy: 0.7878 - loss: 0.5542

12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 637ms/step - accuracy: 0.7887 - loss: 0.5540 - val_accuracy: 0.7604 - val_loss: 0.6906
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - accuracy: 0.8222 - loss: 0.4888

12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 508ms/step - accuracy: 0.8213 - loss: 0.4906 - val_accuracy: 0.7708 - val_loss: 0.6598
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step - accuracy: 0.7984 - loss: 0.5225

12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 633ms/step - accuracy: 0.8001 - loss: 0.5220 - val_accuracy: 0.7812 - val_loss: 0.6776
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 483ms/step - accuracy: 0.8241 - loss: 0.4435 - val_accuracy: 0.7396 - val_loss: 0.7049
Epoch 9/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step - accuracy: 0.8180 - loss: 0.4863

12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 634ms/step - accuracy: 0.8156 - loss: 0.4870 - val_accuracy: 0.8021 - val_loss: 0.6225
Epoch 10/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 485ms/step - accuracy: 0.8481 - loss: 0.4222 - val_accuracy: 0.7812 - val_loss: 0.6729


In [35]:
!ls


best_model.h5  dataset	drive  sample_data


In [36]:
!pip install tf2onnx


INFO: pip is looking at multiple versions of onnx to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.3 requires protobuf>=4.25.2; python_ver

In [1]:
!python -m tf2onnx.convert --keras best_model.h5 --output wafer_model.onnx


2026-01-29 09:54:43.700271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769680483.720182   13693 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769680483.726276   13693 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769680483.742260   13693 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769680483.742285   13693 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769680483.742288   13693 computation_placer.cc:177] computation placer alr

In [2]:
model.save("saved_model")


NameError: name 'model' is not defined

In [3]:
from tensorflow.keras.models import load_model

model = load_model("best_model.h5")



In [4]:
!python -m tf2onnx.convert --saved-model saved_model --output wafer_model.onnx


2026-01-29 09:57:49.059455: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769680669.078628   14521 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769680669.084474   14521 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769680669.099139   14521 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769680669.099167   14521 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769680669.099171   14521 computation_placer.cc:177] computation placer alr

In [5]:
!ls



best_model.h5  dataset	drive  sample_data


In [6]:
from tensorflow.keras.models import load_model

model = load_model("best_model.h5")

model.export("saved_model")


Saved artifact at 'saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_5')]
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  136744054944592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054944016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054944208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054943632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054945168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054941328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054944784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054944976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054942096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054946128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054945

In [7]:
!ls



best_model.h5  dataset	drive  sample_data  saved_model


In [8]:
model.export()



TypeError: Model.export() missing 1 required positional argument: 'filepath'

In [9]:
model.export("saved_model")


Saved artifact at 'saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_5')]
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  136744054944592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054944016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054944208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054943632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054945168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054941328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054944784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054944976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054942096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054946128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136744054945

In [10]:
!python -m tf2onnx.convert --saved-model saved_model --output wafer_model.onnx


2026-01-29 10:01:24.049195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769680884.071193   15440 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769680884.081081   15440 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769680884.103299   15440 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769680884.103329   15440 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769680884.103335   15440 computation_placer.cc:177] computation placer alr

In [1]:
!pip install numpy==1.26.4



In [1]:
from tensorflow.keras.models import load_model

model = load_model("best_model.h5")

model.export("saved_model")


Saved artifact at 'saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_5')]
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  136690468724496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136690468725072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136690468726800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136690468726416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136690468725264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136690468726992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136690468725456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136690468725840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136690468725648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136690468726224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136690468727

In [2]:
!python -m tf2onnx.convert --saved-model saved_model --output wafer_model.onnx


2026-01-29 10:03:56.530555: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769681036.549768   16273 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769681036.555939   16273 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769681036.576864   16273 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769681036.576896   16273 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769681036.576916   16273 computation_placer.cc:177] computation placer alr

In [3]:
!ls


best_model.h5  dataset	drive  sample_data  saved_model  wafer_model.onnx
